In [1]:
import importlib, sys
sys.path.append(r"C:\Users\bong2\OneDrive\Python_Codes\00_python_scripts") # script 경로 window용

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 100)  

import pandas as pd
import requests
from bs4 import BeautifulSoup
import conn_db
import helper
user_agent = helper.user_agent

In [100]:
importlib.reload(stock_fnguide_cleaner)

<module 'stock_fnguide_cleaner' from 'C:\\Users\\bong2\\OneDrive\\Python_Codes\\00_python_scripts\\stock_fnguide_cleaner.py'>

In [6]:
import time

In [5]:
import stock_fnguide_updater, stock_fnguide_cleaner

In [8]:
new = conn_db.from_("DB_기업정보", 'FS_update_list')['종목코드']
old = conn_db.from_("DB_기업정보", 'from_fnguide_기업정보')['종목코드']
new_code_list = list(set(stock_fnguide_updater.code_list) - set(old))

In [2]:
code_list = conn_db.from_("DB_기업정보", 'FS_update_list')['종목코드']

In [3]:
old = conn_db.from_("DB_기업정보", 'from_fnguide_기업정보')['종목코드']
new_code_list = list(set(code_list) - set(old))

In [4]:
# fnguide 기업정보 가져올때 결과물 넣을 dataframes
company_info = pd.DataFrame() # 업종, business summary
financial_highlights = pd.DataFrame() # 재무제표
sales_mix = pd.DataFrame() # 제품별 매출비중 가장 최근
market_share = pd.DataFrame() # 시장점유율 가장 최근
cogs_n_oc = pd.DataFrame() # 판관비율추이, 매출원가율추이
export_n_domestic = pd.DataFrame() # 수출 및 내수 구성

In [11]:
for code in new_code_list:
    try:
        _get_fnguide_company_info(code)
        time.sleep(2)
    except:
        pass

357120 완료
350520 완료
338100 완료
330590 완료
395400 완료
380440 완료
377190 완료
348950 완료
293940 완료
357250 완료
381970 완료
334890 완료


In [25]:
'KRX' in company_info.columns.tolist()

True

In [18]:
company_info['KRX'] = None

In [20]:
company_info['KRX'] =company_info['KRX'].fillna(company_info['KSE'])

In [16]:
r = requests.get(url, headers={'User-Agent': user_agent})
dom = BeautifulSoup(r.text, "html.parser")

In [18]:
temp = []
for i in [1,2]: # 1=KSE 업종, 2=FICS 업종
    text = dom.select(f'#compBody > div.section.ul_corpinfo > div.corp_group1 > p > span.stxt.stxt{i}')[0]
    temp.append(text.text.replace('\xa0',' ').split(' ',1))
industry = pd.DataFrame(temp).T
# 첫행을 컬럼으로 변경하고 원래있던 첫행은 삭제
industry.columns = industry.iloc[0]
industry = industry.iloc[1:,].copy().reset_index(drop=True)

In [19]:
industry

,KONEX,FICS
0,코스닥 제조,식료품


In [20]:
try: # 일부 종목은 Business Summary 없음
    biz_summary_date = dom.select('#bizSummaryDate')[0].text.strip()[1:-1]
    # 제목
    biz_summary_title = dom.select('#bizSummaryHeader')[0].text.replace('\xa0',' ').strip()
    # 내용
    contents =[]
    for i in range(len(dom.select('#bizSummaryContent > li'))):
        contents.append(dom.select('#bizSummaryContent > li')[i].text.replace('\xa0',' ').strip())
    contents = [contents[0] + " " + contents[1]]
    # 합쳐서 하나로 만들기 + 업종도
    contents = pd.DataFrame([biz_summary_date, biz_summary_title, contents]).T
    contents[2] = contents[2][0][0] # 내용 컬럼의 값이 앞뒤로 list[]화 되어 있어서 문자열로 변경
    df = pd.concat([contents, industry], axis=1)
    df['종목코드'] = code
    # 결과물 추가
    company_info = company_info.append(df)
    del contents, biz_summary_date, biz_summary_title, industry, temp
except:
    pass

In [15]:
company_info = helper.make_keycode(company_info.reset_index(drop=True))

In [21]:
if code == '260970':
    try:
        company_info.rename(columns={'KONEX':'KOSDAQ', inplace=True}
    except:
        pass

,0,1,2,KONEX,FICS,종목코드
0,2021/10/02,신사업을 통한 매출 성장,"농축산물과 동물성 원료로부터 분말형태로 식품첨가물을 추출하여 조미식품,기타 식품에 ...",코스닥 제조,식료품,260970


In [17]:
all_cols = company_info.columns.tolist()

In [18]:
for col in all_cols: # 컬럼별로 앞뒤 공백제거
    company_info[col] = company_info[col].str.strip()
    
if 'KSE' in all_cols:
    company_info['KRX'] = (company_info['KSE'].astype(str) + company_info['KOSDAQ'].astype(str)).str.replace('nan','').str.strip()
else:
    company_info['KRX'] = company_info['KOSDAQ'].astype(str).str.replace('nan','').str.strip()
# 종목앞에 코스피/코스닥이 있어서 삭제
company_info['KRX'] = company_info['KRX'].str.split(' ',1,expand=True)[1] 
company_info['KRX'].fillna(company_info['FICS'], inplace=True)

KeyError: 'KOSDAQ'

In [19]:
company_info

,0,1,2,KSE,FICS,종목코드,종목명,KEY
0,2021/05/10,국내 및 해외의 고객사 확보,"SK이노베이션 주식회사의 소재사업부문을 단순ㆍ물적분할하여 설립되었으며, 배터리 소재...","코스피 전기,전자",전자 장비 및 기기,361610,SK아이이테크놀로지,KOSPI361610


In [10]:
def _get_fnguide_company_info(code):
    # fnguide 기업정보 결과물 넣을 dataframes---------------
    global company_info # 업종, business summary
    global financial_highlights # 재무제표
    global sales_mix # 제품별 매출비중 가장 최근
    global market_share # 시장점유율 가장 최근
    global cogs_n_oc # 판관비율추이, 매출원가율추이
    global export_n_domestic # 수출 및 내수 구성

    # 업종, business summary, 재무제표 공통
    url = f"http://comp.fnguide.com/SVO2/asp/SVD_Main.asp?pGB=1&gicode=A{code}&cID=&MenuYn=Y&ReportGB=&NewMenuID=101&stkGb=701"
    time.sleep(2)
    r = requests.get(url, headers={'User-Agent': user_agent})
    dom = BeautifulSoup(r.text, "html.parser")

    # 업종, business summary
    # 업종 가져오기
    temp = []
    for i in [1,2]: # 1=KSE 업종, 2=FICS 업종
        text = dom.select(f'#compBody > div.section.ul_corpinfo > div.corp_group1 > p > span.stxt.stxt{i}')[0]
        temp.append(text.text.replace('\xa0',' ').split(' ',1))
    industry = pd.DataFrame(temp).T
    # 첫행을 컬럼으로 변경하고 원래있던 첫행은 삭제
    industry.columns = industry.iloc[0]
    industry = industry.iloc[1:,].copy().reset_index(drop=True)
    #-------------------------------------

    # Business Summary 가져오기
    # update날짜 앞뒤로 []가 있어서 삭제
    try: # 일부 종목은 Business Summary 없음
        biz_summary_date = dom.select('#bizSummaryDate')[0].text.strip()[1:-1]
        # 제목
        biz_summary_title = dom.select('#bizSummaryHeader')[0].text.replace('\xa0',' ').strip()
        # 내용
        contents =[]
        for i in range(len(dom.select('#bizSummaryContent > li'))):
            contents.append(dom.select('#bizSummaryContent > li')[i].text.replace('\xa0',' ').strip())
        contents = [contents[0] + " " + contents[1]]
        # 합쳐서 하나로 만들기 + 업종도
        contents = pd.DataFrame([biz_summary_date, biz_summary_title, contents]).T
        contents[2] = contents[2][0][0] # 내용 컬럼의 값이 앞뒤로 list[]화 되어 있어서 문자열로 변경
        df = pd.concat([contents, industry], axis=1)
        # fnguide에 오류있어서 수정
        if code == '260970':
            try:
                company_info.rename(columns={'KONEX':'KOSDAQ'}, inplace=True)
            except:
                pass
        df['종목코드'] = code
        # 결과물 추가
        company_info = company_info.append(df)
    except:
        pass
    # 재무제표
    # highlight_D_A # 연결 전체, highlight_B_A # 별도 전체 - 사용x
    df_types = {'highlight_D_Y':'연결 연간',
                'highlight_D_Q':'연결 분기',
                'highlight_B_Y':'별도 연간',
                'highlight_B_Q':'별도 분기'}
    df_all = pd.DataFrame()
    for fs_type in df_types.keys():
        try:
            # 날짜 컬럼
            temp = dom.select(f'#{fs_type} > table > thead > tr.td_gapcolor2 > th > div')
            date_cols = []
            for x in [x.text.strip() for x in temp]:
                if len(x)>7:
                    date_cols.append(x.split('추정치')[-1].strip())
                else:
                    date_cols.append(x)
            # 계정명 컬럼
            temp = dom.select(f'#{fs_type} > table > tbody > tr > th')
            account_cols = []
            for acc in [x.text.strip() for x in temp]:
                if acc.count("(",)>1:
                    account_cols.append(acc.split(')',1)[0] + ')')
                else :
                    account_cols.append(acc)
            df = pd.DataFrame(account_cols)
            # data 값
            data_values = [x.text.strip() for x in dom.select(f'#{fs_type} > table > tbody > tr > td')]
            lap = len(date_cols)
            for i in range(lap):
                temp_df = pd.DataFrame(data_values[i::lap]) # 컬럼의 갯수(lap)만큼 매n번째마다 합쳐줌
                df = pd.concat([df, temp_df], axis=1)
            # 항목 컬럼명 추가해서 df 컬럼 만들기
            df.columns = ['항목'] + date_cols
            df['연결/별도'] = df_types[fs_type].split(' ')[0]
            df['연간/분기'] = df_types[fs_type].split(' ')[1]
            df = df.melt(id_vars=['항목', '연결/별도', '연간/분기'], var_name='날짜',value_name='값')
            df = helper.clean_numeric_value(df)
            df_all = df_all.append(df)
        except:
            pass
    try:
        df_all['종목코드'] = code
        # 결과물 추가
        financial_highlights = financial_highlights.append(df_all)
        del df_all, df, temp, data_values
    except:
        pass

    #----- 제품별 매출비중, 시장점유율, 판관비율, 매출원가율, 수출 및 내수구성 공통 ------ ------ ------
    url = f"http://comp.fnguide.com/SVO2/asp/SVD_Corp.asp?pGB=1&gicode=A{code}&cID=&MenuYn=Y&ReportGB=&NewMenuID=102&stkGb=701"
    r = requests.get(url, headers={'User-Agent': user_agent})
    dom = BeautifulSoup(r.text, "html.parser")

    #----- 제품별 매출비중 가장 최근 ----- ------ ------ ------ ------ ------ ------ ------ -----
    # 날짜 컬럼
    date_cols = dom.select('#divProduct > div.ul_col2_l > div > div.um_table.pd_t1 > table > thead > tr > th')
    date_cols = [x.text.strip() for x in date_cols]

    # 제품명
    products = dom.select('#divProduct > div.ul_col2_l > div > div.um_table.pd_t1 > table > tbody > tr > th')
    products = [x.text.replace('\xa0',' ').strip() for x in products]
    df = pd.DataFrame(products)

    # 제품별 매출비중
    data_values = dom.select('#divProduct > div.ul_col2_l > div > div.um_table.pd_t1 > table > tbody > tr > td')
    data_values = [x.text.strip() for x in data_values]

    try:
        # 데이터 df
        lap = len(date_cols) -1
        for i in range(lap):
            temp_df = pd.DataFrame(data_values[i::lap]) # 컬럼의 갯수(lap)만큼 매n번째마다 합쳐줌
            df = pd.concat([df, temp_df], axis=1)
        df.columns=date_cols
        df = df[df.columns.tolist()[::lap]]
        # 마지막 정리
        # 본래 날짜 컬럼인것을 '구성비'로 수정하고 마지막 날짜만 남기기
        mix_date = df.columns.tolist()[-1] # 마지막 날짜 컬럼명
        df.rename(columns={mix_date:'구성비'}, inplace=True)
        df['기준날짜'] = mix_date # 마지막날짜를 기준날짜로 삽입
        # 길이가 0이 안되면 삭제
        filt = df['구성비'].apply(len) > 0
        df = df.loc[filt].copy()
        df['구성비'] = df['구성비'].astype('float')/100
        # 제품명이 기타(계)인 것 보다 아래에 있으면 삭제
        filt = df['제품명']=='기타(계)'
        try:
            df = df[df.index.tolist() < df[filt].index].copy().reset_index(drop=True)
        except:
            pass
        # 결과물 추가
        df['종목코드'] = code
        sales_mix = sales_mix.append(df)
        del df, filt, mix_date, data_values, products, date_cols
    except:
        pass

    #----- 시장점유율 가장 최근 ------ ------ ------ ------ ------ ------ ------ ------ ------
    # 표 컬럼
    ms_col = dom.select('#divProduct > div.ul_col2_r > div > div.um_table.pd_t1 > table > thead > th')
    ms_col = [x.text.strip() for x in ms_col]
    # 제품 컬럼
    products = dom.select('#divProduct > div.ul_col2_r > div > div.um_table.pd_t1 > table > tbody > tr > th')
    products = [x.text.replace('\xa0',' ').strip() for x in products]
    # 제품별 시장점유율
    data_values = dom.select('#divProduct > div.ul_col2_r > div > div.um_table.pd_t1 > table > tbody > tr > td')
    data_values = [x.text.strip() for x in data_values]
    # 한번에 df로 만들기
    df = pd.DataFrame([products, data_values]).T
    try:
        df.columns = ms_col
        # 마지막 정리
        filt1 = df['시장점유율'].apply(len) > 0
        filt2 = df['주요제품']!='전체'
        filt = filt1 & filt2
        df = df.loc[filt].copy()
        df['시장점유율'] = df['시장점유율'].astype('float')/100
        df['종목코드'] = code
    # 결과물 추가
        market_share = market_share.append(df)
        del filt1, filt2, df, data_values, products, ms_col
    except:
        pass

    #----- 판관비율추이, 매출원가율추이 ------ ------ ------ ------ ------ ------ ------ ------
    cost_types = {'panguanD_01':'연결,판관비율', 'panguanB_01':'별도,판관비율',
                    'panguanD_02':'연결,매출원가율', 'panguanB_02':'별도,매출원가율'}
    df = pd.DataFrame()
    for cost_type in cost_types.keys():
        try:
            date_cols = dom.select(f'#{cost_type} > div > div.um_table > table > thead > tr > th')
            date_cols = [x.text.strip() for x in date_cols]
            data_values = dom.select(f'#{cost_type} > div > div.um_table > table > tbody > tr > td')
            data_values = [cost_types[cost_type]] +  [x.text.strip() for x in data_values]
            temp = pd.DataFrame(data_values).T
            temp.columns = date_cols
            df = df.append(temp)
        except:
            pass
    try:
        df = df.melt(id_vars='항목',var_name='날짜',value_name='값')
        df = helper.clean_numeric_value(df)
        df['값'] = df['값']/100
        df['연결/별도'] = df['항목'].str.split(',', expand=True)[0]
        df['항목'] = df['항목'].str.split(',', expand=True)[1]
        df['종목코드'] = code
        # 결과물 추가
        cogs_n_oc = cogs_n_oc.append(df)
        del df, date_cols, data_values, temp
    except:
        del df, date_cols, data_values, temp

    # 수출 및 내수구성 정리
    corp_types = {'corpExport_D':'연결',
                  'corpExport_B':'별도'}
    df = pd.DataFrame()
    for corp_type in corp_types.keys():
        try:
            # 날짜컬럼
            col = dom.select(f'#{corp_type} > table > thead > tr.th2row_f > th')
            col = [x.text.strip() for x in col]
            # ['매출유형', '제품명', '2017/12', '2018/12', '2019/12']
            #col의 결과에서 날짜만 선택 → ['2017/12', '2018/12', '2019/12']
            date_cols = col[2:]
            # 날짜와 내수/수출 combination된 df 만들기
            date_df = pd.DataFrame(['매출유형','제품명'] + [(x,y) for x in date_cols for y in ['내수','수출']])
            #------ ------ ------ ------ ------ ------ ------ ------
            # 매출유형 철럼
            sales_type = dom.select(f'#{corp_type} > table > tbody > tr > td.clf') # 매출유형
            sales_type = [x.text.strip() for x in sales_type]
            # ['제품', '기타', '기타', '']
            #------ ------ ------ ------ ------ ------ ------ ------
            # 제품명
            products = dom.select(f'#{corp_type} > table > tbody > tr > td.l')
            products =  [x.text.replace('\xa0',' ').strip()  for x in products]
            # ['Display 패널', 'LCD, OLED 기술 특허', '원재료,부품 등', '합계']
            # 매출유형+제품명 합친 df
            col_df = pd.DataFrame([sales_type, products]).T
            #------ ------ ------ ------ ------ ------ ------ ------
            # 데이터 정리
            data_values = dom.select(f'#{corp_type} > table > tbody > tr > td.r')
            data_values = [x.text.strip() for x in data_values]
            lap = len(date_cols)*2 # 다중index 컬럼이라서 *2 (날짜별 내수/수출)
            temp_df = pd.DataFrame()
            for i in range(lap):
                temp = pd.DataFrame(data_values[i::lap])
                temp_df = pd.concat([temp_df, temp], axis=1)
            #------ ------ ------ ------ ------ ------ ------ ------
            # 매출유형+제품명 합친 df + 데이터 df
            temp_df = pd.concat([col_df, temp_df], axis=1)
            # 매출유형+제품명 합친 df + 데이터 df + 날짜df
            temp_df = pd.concat([date_df, temp_df.T.reset_index(drop=True)], axis=1).T
            #------ ------ ------ ------ ------ ------ ------ ------
            # 합쳐진거 전처리
            # 첫행을 column으로 셋팅하고 첫행 삭제
            temp_df.columns = temp_df.iloc[0]
            temp_df = temp_df.iloc[1:,].copy()
            # wide to tidy
            temp_df = temp_df.melt(id_vars=['매출유형','제품명'], var_name='임시', value_name='값')
            temp_df['연결/별도'] = corp_types[corp_type]
            # 컬럼 정리
            df = df.append(temp_df)
        except:
            pass
    try:
        df['날짜'] = df['임시'].str[0]
        df['수출/내수'] = df['임시'].str[1]
        df.loc[df['매출유형'].apply(len)==0,'매출유형'] = df.loc[df['매출유형'].apply(len)==0,'제품명']
        col = ['매출유형','제품명','수출/내수','연결/별도','날짜','값']
        df = df[col].sort_values(by='날짜', ascending=False)
        '''
        값이 없는 경우에도 제품명은 가져오기 위해서
        df['값'].sum() ==''으로 test해서 각 경우 별도로 처리
        '''
        df = df.drop_duplicates(subset=['매출유형','제품명','연결/별도','값'])
        if df['값'].sum() =='':
            df = df.drop_duplicates(subset=['매출유형','연결/별도','제품명'])
            df['값'] = 0
        else:
            filt = df['값'].apply(len)>0
            df = df.loc[filt].copy()
            df['값'] = df['값'].astype('float')/100
        df['종목코드'] = code
        df.reset_index(drop=True, inplace=True)
        # 결과물 추가
        export_n_domestic = export_n_domestic.append(df)
        # del df, col, temp_df, data_values, products, date_cols
    except:
        pass
    print(f'{code} 완료')